In [ ]:
%pip install llama_index
%pip install /path/to/llama_index_graph_stores_agensgraph-0.1.0-py3-none-any.whl

In [2]:
from llama_index.graph_stores.agensgraph import AgensPropertyGraphStore

In [3]:
conf = {
    "database": "test",
    "user": "",
    "password": "",
    "host": "localhost",
    "port": 5432,
}

# If you want to create a new graph, add the argument `create=True` to the constructor
graph_store = AgensPropertyGraphStore(graph_name="", conf=conf, create=True)

In [4]:
from llama_index.core.graph_stores.types import EntityNode, ChunkNode, Relation

### GET

In [7]:
entity = EntityNode(label="PERSON", name="Alice")
chunk = ChunkNode(text="Alice is a software engineer.")
graph_store.upsert_nodes([entity, chunk])

# Get by ID
graph_store.get(ids=[entity.id])
graph_store.get(ids=[chunk.id])

[ChunkNode(label='text_chunk', embedding=None, properties={}, text='Alice is a software engineer.', id_='-4768515847030714664')]

### get_triplets

In [8]:
person = EntityNode(label="PERSON", name="Alice")
city = EntityNode(label="CITY", name="Paris")
graph_store.upsert_nodes([person, city])

visited_relation = Relation(
    source_id=person.id,
    target_id=city.id,
    label="VISITED",
    properties={"year": 2023},
)
graph_store.upsert_relations([visited_relation])

graph_store.get_triplets(entity_names=["Alice"])

[[EntityNode(label='PERSON', embedding=None, properties={'id': 'Alice'}, name='Alice'),
  Relation(label='VISITED', source_id='Alice', target_id='Paris', properties={'year': 2023}),
  EntityNode(label='CITY', embedding=None, properties={'id': 'Paris'}, name='Paris')]]

### get_rel_map()

In [9]:
e1 = EntityNode(label="PERSON", name="Alice")
e2 = EntityNode(label="PERSON", name="Bob")
e3 = EntityNode(label="CITY", name="Paris")
e4 = EntityNode(label="CITY", name="London")
graph_store.upsert_nodes([e1, e2, e3, e4])

r1 = Relation(label="KNOWS", source_id=e1.id, target_id=e2.id)
r2 = Relation(label="VISITED", source_id=e1.id, target_id=e3.id)
r3 = Relation(label="VISITED", source_id=e2.id, target_id=e4.id)
graph_store.upsert_relations([r1, r2, r3])

graph_store.get_rel_map([e1, e2], depth=2)

[[EntityNode(label='PERSON', embedding=None, properties={'name': 'Alice'}, name='Alice'),
  Relation(label='VISITED', source_id='Alice', target_id='Paris', properties={'year': 2023}),
  EntityNode(label='CITY', embedding=None, properties={'name': 'Paris'}, name='Paris')],
 [EntityNode(label='PERSON', embedding=None, properties={'name': 'Bob'}, name='Bob'),
  Relation(label='VISITED', source_id='Bob', target_id='London', properties={}),
  EntityNode(label='CITY', embedding=None, properties={'name': 'London'}, name='London')],
 [EntityNode(label='PERSON', embedding=None, properties={'name': 'Alice'}, name='Alice'),
  Relation(label='KNOWS', source_id='Alice', target_id='Bob', properties={}),
  EntityNode(label='PERSON', embedding=None, properties={'name': 'Bob'}, name='Bob')],
 [EntityNode(label='PERSON', embedding=None, properties={'name': 'Alice'}, name='Alice'),
  Relation(label='VISITED', source_id='Alice', target_id='Paris', properties={'year': 2023}),
  EntityNode(label='CITY', emb

### Vector Query

In [10]:
from llama_index.core.vector_stores.types import VectorStoreQuery

entity1 = EntityNode(
    label="PERSON", name="Alice", properties={"embedding": [0.1, 0.2, 0.3]}
)
entity2 = EntityNode(
    label="PERSON", name="Bob", properties={"embedding": [0.9, 0.8, 0.7]}
)
graph_store.upsert_nodes([entity1, entity2])

# # Query embedding somewhat closer to [0.1, 0.2, 0.3] than [0.9, 0.8, 0.7]
query = VectorStoreQuery(query_embedding=[0.1, 0.2, 0.31], similarity_top_k=2)
graph_store.vector_query(query)

([EntityNode(label='PERSON', embedding=None, properties={}, name='Alice'),
  EntityNode(label='PERSON', embedding=None, properties={}, name='Bob')],
 [0.9998778131472615, 0.8771844461223388])

### Vector Index

In [12]:
# Create a new graph
graph_store = AgensPropertyGraphStore(graph_name="", conf=conf, create=True, vector_dimension=3)

In [13]:
entity1 = EntityNode(
    label="PERSON", name="Alice", properties={"embedding": [0.1, 0.2, 0.3]}
)
entity2 = EntityNode(
    label="PERSON", name="Bob", properties={"embedding": [0.9, 0.8, 0.7]}
)

graph_store.upsert_nodes([entity1, entity2])

In [14]:
from llama_index.core.vector_stores.types import VectorStoreQuery

# # Query embedding somewhat closer to [0.1, 0.2, 0.3] than [0.9, 0.8, 0.7]
query = VectorStoreQuery(query_embedding=[0.1, 0.2, 0.31], similarity_top_k=2)
graph_store.vector_query(query)

([EntityNode(label='PERSON', embedding=None, properties={}, name='Alice'),
  EntityNode(label='PERSON', embedding=None, properties={}, name='Bob')],
 [0.9998778131472615, 0.8771844461223388])